In [ ]:
%matplotlib qt
import mne
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
%matplotlib qt
from eeg_preprocessing import preprocessing
preprocessing(subject_id="zdfy",
                subjects_dir="/Users/payamsadeghishabestari/antinomics_clean_codes/subjects",
                paradigm="gpias",
                site="Zuerich",
                manual_data_scroll=True,
                run_ica=False,
                manual_ica_removal=False,
                eog_correct=True,
                pulse_correct=True,
                resp_correct=False,
                create_report=True,
                saving_dir=None,
                verbose="ERROR")

In [ ]:
import mne
report = mne.open_report("/Users/payamsadeghishabestari/antinomics_clean_codes/subjects/zdfy/EEG/reports/rest_v1.h5")
report.save("/Users/payamsadeghishabestari/Desktop/rest.html", overwrite=True)

In [ ]:
%matplotlib qt
from eeg_processing import run_rs_analysis
run_rs_analysis(
        subject_id="zdfy",
        subjects_dir="/Users/payamsadeghishabestari/antinomics_clean_codes/subjects",
        visit=1,
        event_ids=None,
        source_analysis=True,
        mri=False,
        subjects_fs_dir=None,
        manual_data_scroll=True,
        create_report=True,
        saving_dir=None,
        verbose="ERROR"
        )

In [ ]:
%matplotlib qt
from eeg_processing import run_erp_analysis
run_erp_analysis(
        subject_id="zdfy",
        subjects_dir="/Users/payamsadeghishabestari/antinomics_clean_codes/subjects",
        paradigm="gpias",
        events=None,
        source_analysis=True,
        mri=False,
        subjects_fs_dir=None,
        manual_data_scroll=True,
        create_report=True,
        saving_dir=None,
        verbose="ERROR"
        )

In [ ]:
%matplotlib qt
from eeg_preprocessing import preprocessing
preprocessing(subject_id="501",
                subjects_dir="/Users/payamsadeghishabestari/antinomics_clean_codes/subjects",
                paradigm="rest_v1",
                site="Regensburg",
                manual_data_scroll=True,
                run_ica=True,
                manual_ica_removal=False,
                eog_correct=False,
                pulse_correct=False,
                resp_correct=False,
                create_report=True,
                saving_dir=None,
                verbose="ERROR")

Eyetracking

In [ ]:
import mne
raw = mne.io.read_raw_brainvision("/Users/payamsadeghishabestari/antinomics_clean_codes/subjects/zdfy/EEG/xxxxx/zdfy_xxxxx.vhdr")
mne.read_events(raw)

In [ ]:
raw.annotations

In [ ]:
from mne.io import read_raw_brainvision, read_raw_eyelink
from mne import events_from_annotations
from mne.preprocessing import realign_raw


## load
fname_eeg = "/Users/payamsadeghishabestari/Downloads/alertness_01_H036.vhdr"
fname_et = "/Users/payamsadeghishabestari/Downloads/local_al1H036.asc"

raw_eeg = read_raw_brainvision(fname_eeg, preload=True)
raw_eye = read_raw_eyelink(fname=fname_et, create_annotations=True) 
events_eeg, event_ids_eeg = events_from_annotations(raw_eeg)
events_eye, event_ids_eye = events_from_annotations(raw_eye)

## for now
stim_id_eeg_1, stim_id_eeg_2 = 8, 9
stim_id_eye_1, stim_id_eye_2 = 4, 5

s_raw = events_eeg[(events_eeg[:,2] == stim_id_eeg_1) | (events_eeg[:,2] == stim_id_eeg_2)][:, 0]
s_other = events_eye[(events_eye[:,2] == stim_id_eye_1) | (events_eye[:,2] == stim_id_eye_2)][:, 0]

## realigning
realign_raw(raw=raw_eeg,
            other=raw_eye,
            t_raw=s_raw / raw_eeg.info["sfreq"] - raw_eeg.first_time,
            t_other=s_other / raw_eye.info["sfreq"] - raw_eye.first_time,
            verbose=None)

raw_eye.add_channels([raw_eeg], force_update_info=True)
events, event_dict = events_from_annotations(raw_eye)
del raw_eeg  

In [ ]:
fname_eeg = "/Users/payamsadeghishabestari/Downloads/alertness_01_H036.vhdr"
fname_et = "/Users/payamsadeghishabestari/Downloads/local_al1H036.asc"

raw_eeg = read_raw_brainvision(fname_eeg, preload=True)
raw_eye = read_raw_eyelink(fname=fname_et, create_annotations=True) 

Lets see what is regularity

In [ ]:
import mne

In [ ]:
fname = "/Users/payamsadeghishabestari/antinomics_clean_codes/subjects/dvob/EEG/regularity/dvob_regularity.vhdr"
raw = mne.io.read_raw_brainvision(fname, preload=True)
raw.resample(250)

In [ ]:
events, events_dict = mne.events_from_annotations(raw)

In [ ]:
import mne
epochs_fnames = ["/Users/payamsadeghishabestari/antinomics_clean_codes/subjects/zdfy/EEG/rest_v1/epochs-eo-epo.fif",
                "/Users/payamsadeghishabestari/antinomics_clean_codes/subjects/zdfy/EEG/rest_v1/epochs-ec-epo.fif"]

epochs, eye_labels = [], []
for ep_fname, title in zip(epochs_fnames, ["open", "close"]):
    eps = mne.read_epochs(ep_fname, preload=True)
    eps.drop_bad()
    epochs.append(eps)
    eye_labels += len(eps) * [title]

In [ ]:
epochs = mne.concatenate_epochs(epochs)

In [ ]:
import numpy as np
import mne_features

In [ ]:
def compute_sensor_band_power(epochs):
    """
    """
    freq_bands = {
                    "delta": (0.5, 4),
                    "theta": (4, 8),
                    "alpha": (8, 13),
                    "beta": (13, 30),
                    "lower_gamma": (30, 45),
                    "upper_gamma": (45, 80)
                    }
    
    psds, freqs = epochs.compute_psd(method="multitaper",
                                    picks="eeg",
                                    fmin=freq_bands["delta"][0],
                                    fmax=freq_bands["upper_gamma"][1],
                                    ).get_data(return_freqs=True)
    psds /= np.sum(psds, axis=-1, keepdims=True)  # normalize the PSDs
    ep_psds = []
    for fmin, fmax in freq_bands.values():
        psds_band = psds[:, :, (freqs >= fmin) & (freqs < fmax)].mean(axis=-1)
        ep_psds.append(psds_band)
    

    return np.array(ep_psds).transpose(1, 2, 0)

In [ ]:
freq_bands = {
                "delta": (0.5, 4),
                "theta": (4, 8),
                "alpha": (8, 13),
                "beta": (13, 30),
                "lower_gamma": (30, 45),
                "upper_gamma": (45, 80)
                }



epochs.pick("eeg")
funcs_params = {
                    "pow_freq_bands__freq_bands": freq_bands,
                    "pow_freq_bands__psd_method": "multitaper",
                    "pow_freq_bands__normalize": True
                    } 

powers = mne_features.feature_extraction.extract_features(epochs.get_data(),
                                                epochs.info["sfreq"],
                                                selected_funcs=["pow_freq_bands"],
                                                funcs_params=funcs_params,
                                                ch_names=epochs.info["ch_names"]
                                                )

In [ ]:
n_chs = len(epochs.info["ch_names"])
n_freq_bands = len(freq_bands.keys())
powers_reshaped = powers.reshape(len(epochs), n_chs, n_freq_bands)

In [ ]:
columns = [f"ch_{j}_freq_{k}" for j in epochs.info["ch_names"] for k in freq_bands]

In [ ]:
df = pd.DataFrame(powers_reshaped, columns=columns)

In [ ]:
hurst = mne_features.feature_extraction.extract_features(epochs.get_data(),
                                                epochs.info["sfreq"],
                                                selected_funcs=["spect_slope"],
                                                funcs_params=None,
                                                ch_names=epochs.info["ch_names"]
                                                )

In [ ]:
y = hurst.reshape(len(epochs), n_chs, 4)[:,:,:2] # intercept and slope

In [ ]:
y.shape

In [ ]:
x = hurst.reshape(len(epochs), n_chs, 4)[:,:,:2]

In [ ]:
x = x.reshape(257, 118)

In [ ]:
x == hurst[:, :118]

In [ ]:
freq_bands = {
            "delta": (0.5, 4),
            "theta": (4, 8),
            "alpha": (8, 13),
            "beta": (13, 30),
            "lower_gamma": (30, 45),
            "upper_gamma": (45, 80)
            }

psds, freqs = epochs.compute_psd(method="multitaper",
                                    picks="eeg",
                                    fmin=freq_bands["delta"][0],
                                    fmax=freq_bands["upper_gamma"][1],
                                    ).get_data(return_freqs=True)

In [ ]:
psds.shape

In [ ]:
X